In [2]:
import numpy as np
import shap
import pandas as pd
from sklearn.linear_model import LogisticRegression
from helper import *
from helper_dep import *
from helper_indep import *
from helper_shapley_sampling import *
from helper_kshap import *
import matplotlib.pyplot as plt

/Users/jeremygoldwasser/opt/anaconda3/envs/shap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


German Credit dataset has 1000 samples, 20 covariates; response is "good customer" (binary); ~70% of Ys are 1. On UCI ML Repo.

Problem: can't load as already categorical. The Gradient Boosting model they used allowed you to just input the (numerical) data w/ a list of the categorical indices -- we can't do that with sklrean logistic regression.
- FWIW, I think we had to convert things manually for the bank dataset. I just don't want to have to deal with that again.
- Actually, it might have been OK to begin with - I just made things more complicated than necessary. Wouldn't be the first time.

In [3]:
# import sage 
# df = sage.datasets.credit()

Census dataset has 30k samples, 12 covariates, binary response, ~75% of Ys are False; some features numerical, some categorical.

From UCI ML Repository. Predict whether income exceeds $50K/yr based on census data.

In [4]:
X, y = shap.datasets.adult()
X_display, y_display = shap.datasets.adult(display=True)

print(X.shape)
X.head()
X_display.head() # No NAs
print(np.unique(y_display, return_counts=True))
X_binarized = pd.get_dummies(X_display)
print(X_binarized.shape)

(32561, 12)
(array([False,  True]), array([24720,  7841]))
(32561, 91)


Make dictionary whose keys are all the original columns, & whose values are lists of all (could just be 1) the columns in the binarized dataset with those features.
- This will be useful when we get around to fitting the SHAP values

In [5]:
mapping_dict = {}
for i, col in enumerate(X_display.columns):
    bin_cols = []
    for j, bin_col in enumerate(X_binarized.columns):
        if bin_col.startswith(col):
            bin_cols.append(j)
    mapping_dict[i] = bin_cols
# mapping_dict
# print(np.sum(np.sum(X_train[:, 14:21], axis=1) != 1)) # Sanity check: It's right

Rescale covariates to be between 0 and 1

In [6]:
X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
y_int = y_display.astype("int8")

# Split into training & test sets

In [7]:
np.random.seed(1)
n, d_bin = X_norm.shape
n_train = round(n*0.75)
train_idx = np.random.choice(n, size=n_train, replace=False)
# X_train, y_train = X_norm.iloc[train_idx], y_int[train_idx]
X_train_pd, y_train = X_norm.iloc[train_idx], y_int[train_idx]
X_train = X_train_pd.to_numpy()

test_idx = np.setdiff1d(list(range(n)), train_idx)
X_test_pd, y_test = X_norm.iloc[test_idx], y_int[test_idx]
X_test = X_test_pd.to_numpy()

#### Train logistic regression model

85% test accuracy, compared w/ 75% class imbalance. So model is pretty good.

In [8]:
logreg = LogisticRegression(max_iter=1000).fit(X_train, y_train)
print(round(np.mean(logreg.predict(X_test)==y_test)*100))

84


### Recreate logreg model in numpy
## NOTE: Previous iterations didn't have (or need) an intercept term

In [9]:
BETA = logreg.coef_.reshape(d_bin)
INTERCEPT = logreg.intercept_

def model(x):
    yhat = sigmoid(np.dot(x, BETA) + INTERCEPT)
    return yhat.item() if x.shape[0]==1 else yhat

xloc = X_test[0:1]
print(logreg.predict_proba(X_test[0:1]))
print(model(xloc)) # Yes, our function matches sklearn
print(y_test[0]) # Correct classification

[[0.1615208 0.8384792]]
0.8384791974975092
1


### Compute gradient & hessian wrt local x

In [10]:
gradient = logreg_gradient(model, xloc, BETA)
hessian = logreg_hessian(model, xloc, BETA)

# Compute SHAP values, assuming independent features
#### Sanity check: Verify true SHAP values of the quadratic approximation add up to $f(x)-Ef(X)$

In [11]:
feature_means = np.mean(X_train, axis=0)
cov_mat = np.cov(X_train, rowvar=False)

avg_CV_empirical = np.mean(f_second_order_approx(model,X_train, xloc, gradient, hessian))
pred = model(xloc)
exp_CV_sum_empirical = pred - avg_CV_empirical
shap_CV_true_indep = compute_true_shap_cv_indep(xloc, gradient, hessian, feature_means, cov_mat, mapping_dict=mapping_dict)
sum_shap_CV_true = np.sum(shap_CV_true_indep)
print(sum_shap_CV_true)
print(exp_CV_sum_empirical) # Yes, they're extremely close

1.513460422903776
1.5134465875411895


## Shapley Sampling
### Bad/weird. Negative correlations for the important features; positive for unimportant.

In [12]:
np.random.seed(13)
independent_features = True
obj_ss = cv_shapley_sampling(model, X_train, xloc, 
                        independent_features,
                        gradient, hessian,
                        mapping_dict=mapping_dict,
                        M=100, n_samples_per_perm=10) # M is number of permutations
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_ss

order = np.argsort(np.abs(final_ests))[::-1]
print(final_ests[order]) # Final SHAP estimates, ordered
print(np.round(corr_ests[order], 2)) # Correlations
print(np.round(100*(corr_ests**2)[order])) # Variance reductions



[ 0.22056109  0.1186138   0.11724608  0.1138988   0.04962156 -0.03434943
  0.00967933  0.00869379 -0.00772812  0.00543945 -0.00340953  0.00245303]
[-0.27 -0.19 -0.49 -0.39 -0.02  0.45  0.16  0.57  0.78  0.44  0.61  0.45]
[ 7.  4. 24. 15.  0. 20.  3. 32. 61. 19. 37. 20.]


## KernelSHAP
#### Now things look good. 30-50%

In [13]:
np.random.seed(1)
obj_kshap = cv_kshap(model, X_train, xloc, 
            independent_features,
            gradient, hessian,
            mapping_dict=mapping_dict,
            M=1000, n_samples_per_perm=10, 
            var_method='boot', n_boot=250)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_kshap
order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(final_ests[order], 3)) # Final SHAP estimates, ordered
print(np.round(corr_ests[order], 2)) # Correlations
print(np.round(100*(corr_ests**2)[order])) # Variance reductions

[ 0.129  0.077  0.052  0.051  0.042  0.027 -0.018 -0.017  0.009  0.008
  0.002  0.001]
[0.56 0.66 0.72 0.74 0.47 0.7  0.63 0.67 0.64 0.7  0.75 0.55]
[32. 43. 52. 55. 22. 49. 40. 45. 41. 49. 57. 31.]


# Dependent Features
### Recondition covariance

In [19]:
u, s, vh = np.linalg.svd(cov_mat, full_matrices=True)
print(s[0]/s[d_bin-1]) # Conditioning number is essentially infinite
s_max = s[0]
K = 10000 # Desired conditioning number
min_acceptable = s_max/K
s2 = np.copy(s)
s2[s <= min_acceptable] = min_acceptable
cov2 = np.matmul(u, np.matmul(np.diag(s2), vh))
# Basically identical
# print(cov2[:3,:3])
# print(cov_mat[:3,:3])

2.428543530227902e+16


### Generate matrices to estimate true SHAP values
- Takes 1m 15s for 1000 perms. 91x91 matrix so takes a while.

In [20]:

D_matrices = make_all_lundberg_matrices(1000, cov2)

## Shapley Sampling
Variance reductions around mid-90s!

In [25]:
np.random.seed(1)
independent_features = False
shap_CV_true_dep = linear_shap_vals(xloc, D_matrices, feature_means, gradient)
obj_dep = cv_shapley_sampling(model, X_train, xloc,
                    independent_features,
                    gradient,
                    shap_CV_true=shap_CV_true_dep, # Equivalently, can give D_matrices instead
                    M=100,n_samples_per_perm=10,
                    mapping_dict=mapping_dict,
                    cov_mat=cov2)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_dep
order = np.argsort(np.abs(final_ests))[::-1]
print(final_ests[order]) # Final SHAP estimates, ordered
print(np.round(corr_ests[order], 2)) # Correlations
print(np.round(100*(corr_ests**2)[order])) # Variance reductions



covariance is not positive-semidefinite.
covariance is not positive-semidefinite.


[0.1645815  0.10720158 0.04594794 0.04562969 0.04204151 0.02953907
 0.02329357 0.02262578 0.02258273 0.02033402 0.01488499 0.00935861]
[0.96 0.95 0.97 0.99 0.92 0.98 0.98 0.97 0.97 0.98 0.96 0.94]
[93. 89. 94. 97. 84. 95. 97. 94. 93. 96. 93. 88.]


## KernelSHAP
Again, around 90% variance reduction!

In [31]:
%run helper_dep
np.random.seed(1)
independent_features = False
shap_CV_true_dep = linear_shap_vals(xloc, D_matrices, feature_means, gradient)
obj_kshap_dep = cv_kshap(model, X_train, xloc,
                    independent_features,
                    gradient,
                    shap_CV_true=shap_CV_true_dep,
                    M=1000,n_samples_per_perm=10,
                    mapping_dict=mapping_dict,
                    cov_mat=cov2)

final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_kshap_dep
order = np.argsort(np.abs(final_ests))[::-1]
print(final_ests[order]) # Final SHAP estimates, ordered
print(np.round(corr_ests[order], 2)) # Correlations
print(np.round(100*(corr_ests**2)[order])) # Variance reductions



covariance is not positive-semidefinite.


[ 0.08526138  0.08317849  0.02560378  0.01684514 -0.01227594 -0.01215819
  0.01103153  0.00620537  0.00499942  0.00401419  0.00301522  0.00197274]
[0.88 0.92 0.89 0.93 0.95 0.83 0.92 0.91 0.91 0.92 0.94 0.93]
[78. 85. 79. 86. 90. 69. 84. 84. 82. 85. 88. 87.]
